In [1]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0


In [2]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [3]:
os.environ["GROQ_API_KEY"] = "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"

In [5]:
# laoding the document
loader = UnstructuredFileLoader("/content/attention_is_all_you_need.pdf")
documents = loader.load()

In [6]:
print(documents)

[Document(metadata={'source': '/content/attention_is_all_you_need.pdf'}, page_content='3 2 0 2\n\ng u A 2\n\n] L C . s c [\n\n7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and 

In [7]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [8]:
texts = text_splitter.split_documents(documents)

In [9]:
texts

[Document(metadata={'source': '/content/attention_is_all_you_need.pdf'}, page_content='3 2 0 2\n\ng u A 2\n\n] L C . s c [\n\n7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and 

In [10]:
type(texts)

list

In [11]:
len(texts)

26

In [12]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
persist_directory = "doc_db"

In [14]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [15]:
vectordb

In [16]:
# retriever
retriever = vectordb.as_retriever()

In [17]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [18]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7b0b0c68aa40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7b0b0c68b5b0>, model_name='llama-3.1-70b-versatile', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [19]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [39]:
# invoke the qa chain and get a response for user query
query = "how many refrences used in this paper, and tell me which one is 5 best one to read ?"
response = qa_chain.invoke({"query": query})

In [40]:
print(response['result'])

According to the text, there are 37 references used in this paper.

As for the top 5 references to read, I'll provide you with the top 5 references that are most relevant to the paper's topic, which appears to be about neural machine translation and attention mechanisms. Keep in mind that this is subjective, and the importance of each reference may vary depending on the reader's interests.

Here are the top 5 references:

1. **[35] Ilya Sutskever, Oriol Vinyals, and Quoc VV Le. Sequence to sequence learning with neural networks. In Advances in Neural Information Processing Systems, pages 3104–3112, 2014.**

This reference is a seminal work on sequence-to-sequence learning with neural networks, which is a fundamental concept in neural machine translation.

2. **[38] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine translation system: Bridging the gap between human and 

In [41]:
print(response["source_documents"][0].metadata["source"])

/content/attention_is_all_you_need.pdf


In [42]:
# invoke the qa chain and get a response for user query
query = "What is this in this pdf , English Constituency Parsing?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

According to the text, English Constituency Parsing is a task in Natural Language Processing (NLP) that involves parsing the grammatical structure of English sentences. Specifically, it involves identifying the constituent parts of a sentence, such as phrases and clauses, and their relationships to each other.

In the context of the paper, the authors are using the Transformer model to perform English Constituency Parsing on the Wall Street Journal (WSJ) portion of the Penn Treebank dataset. They are evaluating the model's ability to generalize to this task, which is different from the machine translation task that the model was originally designed for.
******************************
Source Document: /content/attention_is_all_you_need.pdf
